In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score

import lab4_hmc as hmc

In [2]:
df_train = pd.read_csv('ee-train.csv')
df_test = pd.read_csv('ee-test.csv')

In [3]:
X_train = np.array(df_train.iloc[:, :-1])
y_train = np.array(df_train.iloc[:, -1])
X_test = np.array(df_test.iloc[:, :-1])
y_test = np.array(df_test.iloc[:, -1])

scaler = StandardScaler() # Standardise input variables

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Change continuous target $y$ into discrete $[0, 1]$ for classifier

In [4]:
# if y > 23.0, positive 1, negative 0 otherwise
for i, y in enumerate(y_train):
    if y > 23.0:
        y_train[i] = 1
    else:
        y_train[i] = 0

for i, y in enumerate(y_test):
    if y > 23.0:
        y_test[i] = 1
    else:
        y_test[i] = 0
        
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [5]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,

In [6]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### energy function using Bernoulli likelihood and sigmoid link function

In [8]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def e_func(w, X, t):
    total = 0
    for i, X_i in enumerate(X):
        pred = w.T@X_i
        sig_pred = sigmoid(pred)
        ans = t[i] * np.log(sig_pred) + (1-t[i]) * np.log(1-sig_pred)
        total += ans
    
    return -total
        
def e_grad(w, X, t):
    ans = np.sum((t-sigmoid(X@w)).reshape(-1, 1)*X, axis=0)
    return -ans

In [9]:
X = X_train
t = y_train

In [10]:
w = np.zeros(8)
print(w)
hmc.gradient_check(w, e_func, e_grad, X, t)

[0. 0. 0. 0. 0. 0. 0. 0.]
Calc.         Numeric       Delta         Acc.
     -146.86       -146.86   2.853218e-08  10
     153.219       153.219   3.354142e-07   9
    -53.1448      -53.1448  -1.457336e-07   9
     175.673       175.673   1.607642e-07  10
    -180.334      -180.334   2.863047e-07   9
      -7.231        -7.231  -1.041556e-07   8
    -16.1591      -16.1591  -1.344711e-07   9
    -11.4549      -11.4549  -5.758758e-08   9


In [11]:
np.random.seed(seed=1)  # For reproducibility
R = 10000
burn = int(R/10)
L = 100  # OK here (should be larger in regression sampling)
eps = 0.009
# Trial-and-error ... feel free to experiment!

S, *_ = hmc.sample(w, e_func, e_grad, R, L, eps, burn=burn, checkgrad=False, args=[X, t])

<ipython-input-8-a6250d6b7f0d>:9: RuntimeWarning: divide by zero encountered in log
  ans = t[i] * np.log(sig_pred) + (1-t[i]) * np.log(1-sig_pred)
<ipython-input-8-a6250d6b7f0d>:9: RuntimeWarning: invalid value encountered in multiply
  ans = t[i] * np.log(sig_pred) + (1-t[i]) * np.log(1-sig_pred)


|----------|  0% accepted [ 50 secs to go ]
|#---------| 96% accepted [ 45 secs to go ]
|##--------| 96% accepted [ 40 secs to go ]
|###-------| 96% accepted [ 35 secs to go ]
|####------| 95% accepted [ 30 secs to go ]
|#####-----| 95% accepted [ 25 secs to go ]
|######----| 95% accepted [ 20 secs to go ]
|#######---| 95% accepted [ 15 secs to go ]
|########--| 95% accepted [ 10 secs to go ]
|#########-| 95% accepted [ 5 secs to go ]
|##########| 95% accepted [ 0 secs to go ]
HMC: R=10000 / L=100 / eps=0.009 / Accept=94.6%


In [12]:
w_mean_opt = np.mean(S, axis=0)
y_test_pred = sigmoid(X_test@w_mean_opt)

In [15]:
w_mean_opt

array([-3.41380133e+00, -8.02626563e+01,  3.86706567e+01,  7.54747848e+01,
        1.05039392e+01,  2.17647135e-01,  7.44493297e+00,  2.40076378e-03])

In [13]:
for i, y in enumerate(y_test_pred):
    if y > 0.5:
        y_test_pred[i] = 1
    else:
        y_test_pred[i] = 0

In [14]:
print("test classification rate =", accuracy_score(y_test, y_test_pred))

test classification rate = 0.9921875
